### Import the required libraries and scrape table from wikipedia

In [7]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

response = get(url)

soup = BeautifulSoup(response.text, "html.parser")
canada_wiki = soup.find('table', class_ = 'wikitable sortable')



#### Find all rows and copy the text to a new dataframe 
After creating the dataframe assign the columns names and clean the dataframe

In [8]:
rows = canada_wiki.findAll('tr')
l = []
for td in rows:
    cols = td.find_all('td')
    row = [col.text for col in cols]
    l.append(row)
c_t = pd.DataFrame(l , columns = ['Postcode' , 'Borough', 'Neighbourhood'])
c_t = c_t.replace('\n','', regex=True)
c_t.head()

,Postcode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


#### Ignoring all cells that are not assigned in Borough

In [10]:
ct = c_t[c_t['Borough'] != 'Not assigned']
ct.drop(ct.index[0], inplace=True)
ct.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


##### reseting the index


In [11]:
ct = ct.reset_index()
ct.head()

,index,Postcode,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,Harbourfront
3,6,M5A,Downtown Toronto,Regent Park
4,7,M6A,North York,Lawrence Heights


###### Droping the index column after reseting the dataframe index

In [12]:
ct.drop(['index'] , axis = 1 , inplace=True)

In [13]:
ct.head(7)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned


##### Assigning the cell that have a Neighbourhood Not assigned with the same as the borough

In [14]:
ct['Neighbourhood'][6] = 'Queen\'s Park'

In [15]:
final_canada =  ct.groupby(['Postcode' , 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

final_canada.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Printing out the shape of the dataframe

In [16]:
final_canada.shape

(103, 3)

## Copy the new geocode csv file into a dataframe

In [2]:
geo = pd.read_csv("Geospatial_Coordinates.csv") 
# Preview the first 5 lines of the loaded data 
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [3]:
geo.rename(columns={'Postal Code': 'Postcode'}, inplace=True)

In [20]:
result = pd.concat([final_canada, geo], axis=1, join='inner')

result.head()

,Postcode,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


## Select Only the unique columns from the previous dataframe

In [23]:
final_result = result.loc[:,~result.columns.duplicated()]

final_result.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
